In [1]:
# %pip install tfx==1.4.0

In [2]:
import os

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [3]:
context = InteractiveContext(pipeline_root='../tfx')

In [5]:
data_dir = "./data"
example_gen = tfx.components.CsvExampleGen(input_base=os.path.join(os.getcwd(), data_dir))

context.run(example_gen)

RuntimeError: Files in same split /home/jupyter/home/repository/git/oonisim/python-programs/tensorflow/tfx/./data/* have different header.

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

In [ ]:
context.show(schema_gen.outputs['schema'])

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

In [ ]:
transform_file = os.path.join(os.getcwd(), '../components/module.py')

In [ ]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

In [ ]:
trainer_file = os.path.join(os.getcwd(), '../components/module.py')

In [ ]:
from tfx.proto import trainer_pb2

TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

trainer = tfx.components.Trainer(
    module_file=trainer_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

### Load TensorBoard

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri

In [ ]:
log_dir = os.path.join(model_artifact_dir, 'logs/')

%load_ext tensorboard
%tensorboard --logdir {log_dir} 

### Evaluate the model

In [ ]:
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
)
context.run(model_resolver)

In [ ]:
# nb it always blesses on first run even if below threshold
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="serving_default",
            label_key="consumer_disputed",
            # preprocessing_function_names=["transform_features"],
        )
    ],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=["product"])],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.65}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10})
                    ),
                ),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
            ],
        )
    ],
)


In [ ]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)
context.run(evaluator)

In [ ]:
#NB TFMA visualizations will not run in Jupyter Lab
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

In [ ]:
tfma.view.render_slicing_metrics(tfma_result)

In [ ]:
from tfx.proto import pusher_pb2

_serving_model_dir = "../interactive-pipeline/serving_model_dir"

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

### Extra stuff

In [ ]:
!mkdir -p ../interactive-pipeline/serving_model_dir

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

# THEN REFRESH BROWSER PAGE!

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

In [ ]:
# Show data sliced by product
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='product')

In [ ]:
# fairness indicators direct from pipeline
# https://colab.research.google.com/github/tensorflow/fairness-indicators/blob/master/fairness_indicators/examples/Fairness_Indicators_Lineage_Case_Study.ipynb